In [1]:
import datetime 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import datetime 
import numpy as np
import requests

In [9]:
###########################
#To get S&P500 ticker######
###########################
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
ticker = df['Symbol'].values
df['Security'] = df['Security'].str.replace('-', '')
security = df['Security'].values

In [11]:
###########################
# Create annual and quarter data frame to keep EPS
###########################
annual=pd.DataFrame()
quarter=pd.DataFrame()

# USE FOR LOOP TO LOOP IN our ticker and company name to loop the URL
for ti, name in zip(ticker, security):
    # Define your header
    headers = {"User-Agent" : "Mozilla/5.0"}
    
    # loop in company link
    url = 'https://www.macrotrends.net/stocks/charts/{}/{}/eps-earnings-per-share-diluted'.format(ti,name)
    # get page content
    page = requests.get(url, allow_redirects=True)
    page_content = page.content
    soup = BeautifulSoup(page_content,"html.parser")
    
    # get ur eps table 
    tabl = soup.find_all("table" , {"class" : "historical_data_table table"})
    
    # assign a blank array to stoy eps
    # here is the start to get annual eps
    eps =[]
    for t in tabl[0:1]:
        temp1 = t.findAll('tr')
        for i in temp1[0:1]:
            value = i.find('th').getText()
        for i in temp1[1:]:
            for j in i:
                eps.append(j.get_text())
    try:
        while True:
            eps.remove('\n')
    except ValueError:
        pass
    
    # store a annual EPS and get year
    eps_value=[]
    year =[]
    for i in eps:
        if (i.find('$')==0):
            eps_value.append(i)
        if (i.find('$')==-1):
            year.append(i)
            
    # store annual in data frame
    Annual_EPS = pd.DataFrame(data= {'year': year, 'eps_value': eps_value})
    Annual_EPS['ticker'] = ti
    annual = pd.concat([annual,Annual_EPS])
    
    # start to scrap for quarter, u will see it below to tabl 2, thus, the index is 1 
    eps =[]
    for t in tabl[1:2]:
        temp1 = t.findAll('tr')
        for i in temp1[0:1]:
            value = i.find('th').getText()
        for i in temp1[1:]:
            for j in i:
                eps.append(j.get_text())

    try:
        while True:
            eps.remove('\n')
    except ValueError:
        pass
    
    # store a quarter EPS and get quarter 
    eps_value=[]
    quarter =[]
    for i in eps:
        if (i.find('$')==0):
            eps_value.append(i)
        if (i.find('$')==-1):
            quarter.append(i)
            
    # store quarter in data frame
    Quarterly_EPS = pd.DataFrame(data= {'Quarterly':quarter, 'eps_value': eps_value})
    Quarterly_EPS['ticker'] = ti
    quarter = pd.concat([quarter, Quarterly_EPS])


In [16]:
quarter

,Quarterly,eps_value,ticker
0,2022-09-30,$6.77,MMM
1,2022-06-30,$0.14,MMM
2,2022-03-31,$2.26,MMM
3,2021-12-31,$2.31,MMM
4,2021-09-30,$2.45,MMM
...,...,...,...
40,2012-09-30,$0.32,ZTS
41,2012-06-30,$0.35,ZTS
42,2012-03-31,$0.22,ZTS
43,2011-12-31,$0.00,ZTS


In [17]:
annual

,year,eps_value,ticker
0,2021,$10.12,MMM
1,2020,$9.36,MMM
2,2019,$7.72,MMM
3,2018,$8.89,MMM
4,2017,$7.93,MMM
...,...,...,...
7,2014,$1.16,ZTS
8,2013,$1.01,ZTS
9,2012,$0.87,ZTS
10,2011,$0.49,ZTS


In [13]:
quarter.to_csv('quarter.csv')

In [15]:
annual.to_csv('annual.csv')